Versão 1.0

In [ ]:
# ==========================================
# Este script realiza a coleta automática de portarias publicadas no Diário Oficial de SC
# para o tema "calculados sobre a média das contribuições".
#
# Fluxo geral:
# 1️⃣ Busca as matérias usando a API:
#     https://portal.doe.sea.sc.gov.br/apis/materia/materia?ano=2025&mes=7&dataInicio=&dataFim=&assunto=35&categoria=4704302&dsMateria=calculados%20sobre%20a%20m%C3%A9dia%20das%20contribui%C3%A7%C3%B5es&tipoBusca=1&sortEdicao=&numeroEdicao=&numeroMateria=&ondePesquisar=4
#
# 2️⃣ Para cada resultado, chama:
#     https://portal.doe.sea.sc.gov.br/apis/edicao-preview/extrato/edicao/4099/materia/1094369
#     Este link retorna a URL final para o PDF específico daquela matéria.
#
# 3️⃣ Baixa o PDF, extrai os dados (nome do servidor, matrícula, cargo, órgão).
# 4️⃣ Gera uma planilha Excel com os resultados.
# ==========================================
!pip install requests pandas pdfplumber tqdm

import requests
import pandas as pd
import pdfplumber
from tqdm import tqdm
import re

anos = range(2025, 2026)
meses = range(1, 13)

BASE_URL = "https://portal.doe.sea.sc.gov.br"
API_MATERIA = BASE_URL + "/apis/materia/materia"
API_PDF_JSON = BASE_URL + "/apis/edicao-preview/extrato/edicao/{}/materia/{}"

HEADERS = {"User-Agent": "Mozilla/5.0"}

dados = []

print("🔎 Buscando matérias…")

# === Etapa 1: Consulta à API para listar as matérias ===
for ano in anos:
    for mes in meses:
        params = {
            "ano": ano, "mes": mes,
            "assunto": "35", "categoria": "4704302",
            "dsMateria": "calculados sobre a média das contribuições",
            "tipoBusca": "1", "ondePesquisar": "4"
        }
        resp = requests.get(API_MATERIA, params=params, headers=HEADERS)
        if resp.ok:
            lista = resp.json()
            if isinstance(lista, list):
                dados.extend(lista)

print(f"✅ {len(dados)} matérias encontradas")

# === Etapa 2: Para cada matéria, pega a URL real do PDF ===
def obter_pdf_url(cdJornal, cdMateria):
    url_json = API_PDF_JSON.format(cdJornal, cdMateria)
    resp = requests.get(url_json, headers=HEADERS)
    if not resp.ok:
        return None
    return resp.json().get("urlExtratoArquivo")

# === Etapa 3: Processa o PDF e devolve lista de registros válidos ===
def extrair_detalhes_pdf(url_pdf):
    resp = requests.get(url_pdf, headers=HEADERS)
    if not resp.ok:
        return []

    with open("/tmp/materia.pdf", "wb") as f:
        f.write(resp.content)

    with pdfplumber.open("/tmp/materia.pdf") as pdf:
        texto = ""
        for page in pdf.pages:
            texto += page.extract_text().replace("\n", " ")

    texto = re.sub(r'\s+', ' ', texto).strip()
    texto = re.sub(r'-\s+(\d+)', r'-\1', texto)  # Corrige matrículas quebradas

    # Quebra o texto em blocos por portaria
    blocos = re.split(r'(PORTARIA\s+Nº\s+\d+\s*-\s*\d{2}/\d{2}/\d{4}\.)', texto)

    registros = []

    for i in range(1, len(blocos), 2):
        cabecalho = blocos[i]
        corpo = blocos[i+1]
        bloco_texto = (cabecalho + " " + corpo).strip()

        if "calculados sobre a média das contribuições" not in bloco_texto.lower():
            continue

        # Número da portaria e data
        try:
            portaria_match = re.search(r'PORTARIA\s+Nº\s+(\d+)\s*-\s*(\d{2}/\d{2}/\d{4})', cabecalho, flags=re.IGNORECASE)
            numero_portaria = portaria_match.group(1)
            data_portaria = portaria_match.group(2)
        except:
            numero_portaria = ""
            data_portaria = ""

        # Nome
        try:
            idx_matricula = bloco_texto.lower().find(", matrícula")
            texto_ate_matricula = bloco_texto[:idx_matricula]
            idx_a = max(
                texto_ate_matricula.rfind(" à "),
                texto_ate_matricula.rfind(" a "),
                texto_ate_matricula.rfind(" de ")
            )
            nome = texto_ate_matricula[idx_a+3:].strip() if idx_a != -1 else ""
        except:
            nome = ""

        # Matrícula (valida formato com 12 caracteres)
        try:
            match_matricula = re.search(
                r'(\d{7}-\d-\d{2})', bloco_texto
            )
            matricula = match_matricula.group(1).strip() if match_matricula else ""
        except:
            matricula = ""

        # Cargo
        try:
            cargo = bloco_texto.split("no cargo de")[-1].split(",")[0].strip()
        except:
            cargo = ""

        # Órgão — última palavra entre '-' e '.'
        try:
          orgao_match = re.search(r'[–-]\s*([A-Z]+)\.', bloco_texto)
          orgao = orgao_match.group(1).strip() if orgao_match else ""
        except:
          orgao = ""


        registros.append((nome, matricula, cargo, orgao, numero_portaria, data_portaria))

    return registros

# === Etapa 4: Processa todas as matérias e coleta todos os registros válidos ===
resultados = []
for item in tqdm(dados, desc="📄 Processando matérias"):
    url_pdf = obter_pdf_url(item["cdJornal"], item["cd_materia"])
    if not url_pdf:
        continue

    registros_pdf = extrair_detalhes_pdf(url_pdf)

    for nome, matricula, cargo, orgao, numero_portaria, data_portaria in registros_pdf:
        resultados.append({
            "Nome do Servidor": nome,
            "Matrícula": matricula,
            "Cargo": cargo,
            "Órgão de Origem": orgao,
            "Data de Publicação do DOE": item["dtPublicacaoJornal"][:10],
            "Número da Edição do DOE": item["vlNumero"],
            "Número da Portaria": numero_portaria,
            "Data da Portaria": data_portaria
        })

# Converte para DataFrame
df_final = pd.DataFrame(resultados)

# === Etapa 5: Salva Excel ===
df_final.to_excel("portarias_doe.xlsx", index=False)
print("📁 Arquivo Excel gerado: portarias_doe.xlsx")

from google.colab import files
files.download("portarias_doe.xlsx")


🔎 Buscando matérias…
✅ 207 matérias encontradas


📄 Processando matérias: 100%|██████████| 207/207 [07:33<00:00,  2.19s/it]

📁 Arquivo Excel gerado: portarias_doe.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

MELHORIAS A SEREM FEITAS NO CÓDIGO

- número de portaria está vindo sempre o primeiro número - OK
- a matrícula algumas estao vindo erradas, talvez o que podemos fazer seja colocar que todas as matrículas tem que ter 12 numeros obrigatóriamente - OK
- órgão de origem ICP-Brasil não está correto (nem sei de onde ele está tirando isso)- OK
- um deles não está vindo o orgão de origem, verificar. - OK

primeiro o script busca esse retorno de api, contendo o ano e o mês de busca
https://portal.doe.sea.sc.gov.br/apis/materia/materia?ano=2025&mes=7&dataInicio=&dataFim=&assunto=35&categoria=4704302&dsMateria=calculados%20sobre%20a%20m%C3%A9dia%20das%20contribui%C3%A7%C3%B5es&tipoBusca=1&sortEdicao=&numeroEdicao=&numeroMateria=&ondePesquisar=4

com o retorno dessa chamada da api é possivel buscar especificamente cada resultado a partir desse link
https://portal.doe.sea.sc.gov.br/apis/edicao-preview/extrato/edicao/4099/materia/1094369

neste link é possivel obter o .pdf da matéria certificada com todo o conteúdo buscado


##Versão 2.0
Traz como resultado os termos
    Nome do Servidor, Matrícula, Cargo, Órgão de Origem, Data de Publicação do DOE, Número da Edição do DOE, Número da Portaria, Data da Portaria,Modalidade de Aposentadoria, Tipo de Aposentadoria/Percentual e Fundamentação Legal.

In [17]:
# ==========================================
# Coleta automática de portarias do Diário Oficial de SC
# para o tema "calculados sobre a média das contribuições"
# ==========================================

!pip install requests pandas pdfplumber tqdm

import requests
import pandas as pd
import pdfplumber
from tqdm import tqdm
import re

anos = range(2025, 2026)
meses = range(1, 13)

BASE_URL = "https://portal.doe.sea.sc.gov.br"
API_MATERIA = BASE_URL + "/apis/materia/materia"
API_PDF_JSON = BASE_URL + "/apis/edicao-preview/extrato/edicao/{}/materia/{}"

HEADERS = {"User-Agent": "Mozilla/5.0"}

dados = []

print("🔎 Buscando matérias…")

# === Etapa 1: Consulta à API para listar as matérias ===
for ano in anos:
    for mes in meses:
        params = {
            "ano": ano, "mes": mes,
            "assunto": "35", "categoria": "4704302",
            "dsMateria": "calculados sobre a média das contribuições",
            "tipoBusca": "1", "ondePesquisar": "4"
        }
        resp = requests.get(API_MATERIA, params=params, headers=HEADERS)
        if resp.ok:
            lista = resp.json()
            if isinstance(lista, list):
                dados.extend(lista)

print(f"✅ {len(dados)} matérias encontradas")

# === Etapa 2: Para cada matéria, pega a URL real do PDF ===
def obter_pdf_url(cdJornal, cdMateria):
    url_json = API_PDF_JSON.format(cdJornal, cdMateria)
    resp = requests.get(url_json, headers=HEADERS)
    if not resp.ok:
        return None
    return resp.json().get("urlExtratoArquivo")

# === Etapa 3: Processa o PDF e devolve lista de registros válidos ===
def extrair_detalhes_pdf(url_pdf):
    resp = requests.get(url_pdf, headers=HEADERS)
    if not resp.ok:
        return []

    with open("/tmp/materia.pdf", "wb") as f:
        f.write(resp.content)

    with pdfplumber.open("/tmp/materia.pdf") as pdf:
        texto = ""
        for page in pdf.pages:
            texto += page.extract_text().replace("\n", " ")

    texto = re.sub(r'\s+', ' ', texto).strip()
    texto = re.sub(r'-\s+(\d+)', r'-\1', texto)  # Corrige matrículas quebradas

    # Quebra o texto em blocos por portaria
    blocos = re.split(r'(PORTARIA\s+Nº\s+\d+\s*-\s*\d{2}/\d{2}/\d{4}\.)', texto)

    registros = []

    for i in range(1, len(blocos), 2):
        cabecalho = blocos[i]
        corpo = blocos[i+1]
        bloco_texto = (cabecalho + " " + corpo).strip()

        if "calculados sobre a média das contribuições" not in bloco_texto.lower():
            continue

        # Número da portaria e data
        try:
            portaria_match = re.search(r'PORTARIA\s+Nº\s+(\d+)\s*-\s*(\d{2}/\d{2}/\d{4})', cabecalho, flags=re.IGNORECASE)
            numero_portaria = portaria_match.group(1)
            data_portaria = portaria_match.group(2)
        except:
            numero_portaria = ""
            data_portaria = ""

        # Nome
        try:
            idx_matricula = bloco_texto.lower().find(", matrícula")
            texto_ate_matricula = bloco_texto[:idx_matricula]
            idx_a = max(
                texto_ate_matricula.rfind(" à "),
                texto_ate_matricula.rfind(" a "),
                texto_ate_matricula.rfind(" de ")
            )
            nome = texto_ate_matricula[idx_a+3:].strip() if idx_a != -1 else ""
        except:
            nome = ""

        # Matrícula
        try:
            match_matricula = re.search(r'(\d{7}-\d-\d{2})', bloco_texto)
            matricula = match_matricula.group(1).strip() if match_matricula else ""
        except:
            matricula = ""

        # Cargo
        try:
            cargo = bloco_texto.split("no cargo de")[-1].split(",")[0].strip()
        except:
            cargo = ""

        # Órgão
        try:
            orgao_match = re.search(r'[–-]\s*([A-Z]+)\.', bloco_texto)
            orgao = orgao_match.group(1).strip() if orgao_match else ""
        except:
            orgao = ""

        # Modalidade: exatamente as palavras em MAIÚSCULAS após CONCEDER
        try:
            modal_match = re.search(r'CONCEDER\s+(([A-ZÁÉÍÓÚÂÊÔÃÕÇ\s]+))', bloco_texto)
            modalidade = modal_match.group(1).strip() if modal_match else ""
        except:
            modalidade = ""

        # Tipo/Percentual: tudo o que vem depois da modalidade até "nos termos"
        try:
            tipo_match = re.search(
                re.escape(modalidade) + r'(.*?)(?=nos termos)',
                bloco_texto,
                flags=re.IGNORECASE
            )
            tipo = tipo_match.group(1).strip(" ,") if tipo_match else ""
        except:
            tipo = ""

        # Percentual
        try:
            # Procura valores como "31,05%" ou "87%"
            percentual_match = re.search(r'(\d{1,3}(?:[\.,]\d{1,2})?)\s*%', tipo)
            if percentual_match:
                percentual = percentual_match.group(1).replace(",", ".")
            else:
                percentual = "100"
        except:
            percentual = "100"


        # Fundamentação legal
        try:
            texto_normalizado = re.sub(r'[^\x00-\x7F]+', '', bloco_texto)

            funda_match = re.search(r'nos termos(.*?),\s*de acordo', texto_normalizado, flags=re.IGNORECASE)
            if funda_match:
                fundamentacao = "nos termos" + funda_match.group(1).strip()
            else:
                funda_match = re.search(r'nos termos(.*?)de acordo', texto_normalizado, flags=re.IGNORECASE)
                if funda_match:
                    fundamentacao = "nos termos" + funda_match.group(1).strip()
                else:
                    funda_match = re.search(r'nos termos do(.*?)de acordo', texto_normalizado, flags=re.IGNORECASE)
                    if funda_match:
                        fundamentacao = "nos termos do" + funda_match.group(1).strip()
                    else:
                        funda_match = re.search(r'nos termos(.*?)(?:de\s+)?acordo', texto_normalizado, flags=re.IGNORECASE)
                        fundamentacao = "nos termos" + funda_match.group(1).strip() if funda_match else ""
        except:
            fundamentacao = ""

        registros.append((
            nome, matricula, cargo, orgao,
            numero_portaria, data_portaria,
            modalidade, tipo, percentual, fundamentacao
        ))

    return registros

# === Etapa 4: Processa todas as matérias e coleta todos os registros válidos ===
resultados = []
for item in tqdm(dados, desc="📄 Processando matérias"):
    url_pdf = obter_pdf_url(item["cdJornal"], item["cd_materia"])
    if not url_pdf:
        continue

    registros_pdf = extrair_detalhes_pdf(url_pdf)

    for (
        nome, matricula, cargo, orgao,
        numero_portaria, data_portaria,
        modalidade, tipo, percentual, fundamentacao
    ) in registros_pdf:
        resultados.append({
            "Nome do Servidor": nome,
            "Matrícula": matricula,
            "Cargo": cargo,
            "Órgão de Origem": orgao,
            "Data de Publicação do DOE": item["dtPublicacaoJornal"][:10],
            "Número da Edição do DOE": item["vlNumero"],
            "Número da Portaria": numero_portaria,
            "Data da Portaria": data_portaria,
            "Modalidade de Aposentadoria": modalidade,
            "Tipo de Aposentadoria/Percentual": tipo,
            "Percentual": percentual,
            "Fundamentação Legal": fundamentacao
        })

# Converte para DataFrame
df_final = pd.DataFrame(resultados)

# === Etapa 5: Salva Excel ===
df_final.to_excel("portarias_doe.xlsx", index=False)
print("📁 Arquivo Excel gerado: portarias_doe.xlsx")

from google.colab import files
files.download("portarias_doe.xlsx")


🔎 Buscando matérias…
✅ 210 matérias encontradas


📄 Processando matérias: 100%|██████████| 210/210 [07:42<00:00,  2.20s/it]

📁 Arquivo Excel gerado: portarias_doe.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Versão 3
O mesmo da versão 2, porém com a seleção de mês e ano feita pelo usuário

In [ ]:
import requests
import pandas as pd
import pdfplumber
from tqdm.notebook import tqdm
import re
from datetime import date
from ipywidgets import Dropdown, Button, HBox, VBox, Output
from IPython.display import display
import os

# ============ INTERFACE DE SELEÇÃO ============

output = Output()

anos_disponiveis = list(range(2022, date.today().year + 1))
meses_disponiveis = {
    1: "Janeiro", 2: "Fevereiro", 3: "Março", 4: "Abril",
    5: "Maio", 6: "Junho", 7: "Julho", 8: "Agosto",
    9: "Setembro", 10: "Outubro", 11: "Novembro", 12: "Dezembro"
}

meses_por_nome = {v: k for k, v in meses_disponiveis.items()}

ano_inicio_w = Dropdown(options=anos_disponiveis, description="Ano Início:")
mes_inicio_w = Dropdown(options=meses_disponiveis.values(), description="Mês Início:")
ano_fim_w = Dropdown(options=anos_disponiveis, description="Ano Fim:")
mes_fim_w = Dropdown(options=meses_disponiveis.values(), description="Mês Fim:")
botao_gerar = Button(description="📄 Gerar Documento", button_style="success")

BASE_URL = "https://portal.doe.sea.sc.gov.br"
API_MATERIA = BASE_URL + "/apis/materia/materia"
API_PDF_JSON = BASE_URL + "/apis/edicao-preview/extrato/edicao/{}/materia/{}"
HEADERS = {"User-Agent": "Mozilla/5.0"}

def obter_pdf_url(cdJornal, cdMateria):
    url_json = API_PDF_JSON.format(cdJornal, cdMateria)
    resp = requests.get(url_json, headers=HEADERS)
    if not resp.ok:
        return None
    return resp.json().get("urlExtratoArquivo")

def extrair_detalhes_pdf(url_pdf):
    registros = []

    try:
        resp = requests.get(url_pdf, headers=HEADERS)
        if not resp.ok:
            return []

        with open("/tmp/materia.pdf", "wb") as f:
            f.write(resp.content)

        with pdfplumber.open("/tmp/materia.pdf") as pdf:
            texto = ""
            for page in pdf.pages:
                texto += page.extract_text().replace("\n", " ")

        texto = re.sub(r'\s+', ' ', texto).strip()
        texto = re.sub(r'-\s+(\d+)', r'-\1', texto)

        blocos = re.split(r'(PORTARIA\s+Nº\s+\d+\s*-\s*\d{2}/\d{2}/\d{4}\.)', texto)

        for i in range(1, len(blocos), 2):
            cabecalho = blocos[i]
            corpo = blocos[i+1]
            bloco_texto = (cabecalho + " " + corpo).strip()

            if "calculados sobre a média das contribuições" not in bloco_texto.lower():
                continue

            try:
                portaria_match = re.search(r'PORTARIA\s+Nº\s+(\d+)\s*-\s*(\d{2}/\d{2}/\d{4})', cabecalho, flags=re.IGNORECASE)
                numero_portaria = portaria_match.group(1)
                data_portaria = portaria_match.group(2)
            except:
                numero_portaria = ""
                data_portaria = ""

            try:
                idx_matricula = bloco_texto.lower().find(", matrícula")
                texto_ate_matricula = bloco_texto[:idx_matricula]
                idx_a = max(
                    texto_ate_matricula.rfind(" à "),
                    texto_ate_matricula.rfind(" a "),
                    texto_ate_matricula.rfind(" de ")
                )
                nome = texto_ate_matricula[idx_a+3:].strip() if idx_a != -1 else ""
            except:
                nome = ""

            try:
                match_matricula = re.search(r'(\d{7}-\d-\d{2})', bloco_texto)
                matricula = match_matricula.group(1).strip() if match_matricula else ""
            except:
                matricula = ""

            try:
                cargo = bloco_texto.split("no cargo de")[-1].split(",")[0].strip()
            except:
                cargo = ""

            try:
                orgao_match = re.search(r'[–-]\s*([A-Z]+)\.', bloco_texto)
                orgao = orgao_match.group(1).strip() if orgao_match else ""
            except:
                orgao = ""

            try:
                modal_match = re.search(r'CONCEDER\s+(([A-ZÁÉÍÓÚÂÊÔÃÕÇ\s]+))', bloco_texto)
                modalidade = modal_match.group(1).strip() if modal_match else ""
            except:
                modalidade = ""

            try:
                tipo_match = re.search(
                    re.escape(modalidade) + r'(.*?)(?=nos termos)',
                    bloco_texto,
                    flags=re.IGNORECASE
                )
                tipo = tipo_match.group(1).strip(" ,") if tipo_match else ""
            except:
                tipo = ""

            try:
                percentual_match = re.search(r'(\d{1,3}(?:[\.,]\d{1,2})?)\s*%', tipo)
                if percentual_match:
                    percentual = percentual_match.group(1).replace(",", ".")
                else:
                    percentual = "100"
            except:
                percentual = "100"

            try:
                texto_normalizado = re.sub(r'[^\x00-\x7F]+', '', bloco_texto)
                funda_match = re.search(r'nos termos(.*?),\s*de acordo', texto_normalizado, flags=re.IGNORECASE)
                if funda_match:
                    fundamentacao = "nos termos" + funda_match.group(1).strip()
                else:
                    funda_match = re.search(r'nos termos(.*?)de acordo', texto_normalizado, flags=re.IGNORECASE)
                    if funda_match:
                        fundamentacao = "nos termos" + funda_match.group(1).strip()
                    else:
                        funda_match = re.search(r'nos termos do(.*?)de acordo', texto_normalizado, flags=re.IGNORECASE)
                        if funda_match:
                            fundamentacao = "nos termos do" + funda_match.group(1).strip()
                        else:
                            funda_match = re.search(r'nos termos(.*?)(?:de\s+)?acordo', texto_normalizado, flags=re.IGNORECASE)
                            fundamentacao = "nos termos" + funda_match.group(1).strip() if funda_match else ""
            except:
                fundamentacao = ""

            registros.append((
                nome, matricula, cargo, orgao,
                numero_portaria, data_portaria,
                modalidade, tipo, percentual, fundamentacao
            ))

    except:
        pass

    return registros

def executar_coleta(b):
    output.clear_output()
    with output:
        ano_i = int(ano_inicio_w.value)
        mes_i = meses_por_nome[mes_inicio_w.value]
        ano_f = int(ano_fim_w.value)
        mes_f = meses_por_nome[mes_fim_w.value]

        print(f"🔍 Buscando matérias de {mes_inicio_w.value}/{ano_i} até {mes_fim_w.value}/{ano_f}...")

        dados = []
        for ano in range(ano_i, ano_f + 1):
            for mes in range(1, 13):
                if (ano == ano_i and mes < mes_i) or (ano == ano_f and mes > mes_f):
                    continue

                params = {
                    "ano": ano, "mes": mes,
                    "assunto": "35", "categoria": "4704302",
                    "dsMateria": "calculados sobre a média das contribuições",
                    "tipoBusca": "1", "ondePesquisar": "4"
                }
                resp = requests.get(API_MATERIA, params=params, headers=HEADERS)
                if resp.ok:
                    lista = resp.json()
                    if isinstance(lista, list):
                        dados.extend(lista)

        print(f"✅ {len(dados)} matérias encontradas")

        resultados = []
        for item in tqdm(dados, desc="📄 Processando matérias"):
            url_pdf = obter_pdf_url(item["cdJornal"], item["cd_materia"])
            if not url_pdf:
                continue

            registros_pdf = extrair_detalhes_pdf(url_pdf)

            for (
                nome, matricula, cargo, orgao,
                numero_portaria, data_portaria,
                modalidade, tipo, percentual, fundamentacao
            ) in registros_pdf:
                resultados.append({
                    "Nome do Servidor": nome,
                    "Matrícula": matricula,
                    "Cargo": cargo,
                    "Órgão de Origem": orgao,
                    "Data de Publicação do DOE": item["dtPublicacaoJornal"][:10],
                    "Número da Edição do DOE": item["vlNumero"],
                    "Número da Portaria": numero_portaria,
                    "Data da Portaria": data_portaria,
                    "Modalidade de Aposentadoria": modalidade,
                    "Tipo de Aposentadoria/Percentual": tipo,
                    "Percentual": percentual,
                    "Fundamentação Legal": fundamentacao
                })

        df_final = pd.DataFrame(resultados)
        df_final.to_excel("portarias_doe.xlsx", index=False)
        print("📁 Arquivo Excel gerado: portarias_doe.xlsx")
        display(df_final.head())

botao_gerar.on_click(executar_coleta)
display(VBox([HBox([ano_inicio_w, mes_inicio_w]), HBox([ano_fim_w, mes_fim_w]), botao_gerar, output]))
